In [195]:
import sys
sys.path.insert(0, '../')
import utils

IndentationError: unexpected indent (utils.py, line 12)

In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import *

In [2]:
def redistribute_gene_score(coef, mat, signal, out): # TODO: his may be better in the R script for the package
    '''
    calculate the redistributed gene score after the signal on genes was redistributed by regression
    :param coef: path to input file, a data frame, with dims [n_features, n_lambdas]
    :param mat: a binary sparse matrix in text format, with dims [n_genes, n_features]
    :param signal: a [n_genes, ] vector of the original signal on gene
    :return: a [n_lambda, ] list of sparse matrices each has dims [n_genes, n_features] . where entries describe the "explanatory power" of a gene in a feature
    '''
    mat_feature_lambda = np.array(pd.read_table(coef, index_col=0))
    coo_gene_feature = np.loadtxt(mat).astype(int)
    vsignal = np.loadtxt(signal)
    with open(out, 'w') as ofh:
        for i in range(mat_feature_lambda.shape[1]):
            mat_gene_feature = coo_matrix((mat_feature_lambda[:, i][coo_gene_feature[:, 1]],
                                 (coo_gene_feature[:,0], coo_gene_feature[:,1])))
            # rescale this
            for x, y, d in zip(mat_gene_feature.row, mat_gene_feature.col, mat_gene_feature.data):
                if (d == 0) or (vsignal[x]==0):
                    continue
                ofh.write('{}\t{}\t{}\t{:.6f}\n'.format(i, x, y, d * vsignal[x] / float(np.sum(mat_gene_feature, axis=1)[x])))

In [3]:
coef = 'nest1.blca_mutsig_diff_nneg_log_ps5.glmnet-ms-impact.coef'
mat = 'nest1_conn.txt'
signal = 'build_20181101_blca_diff_nneg_log_ps5.txt'
out = 'redistribute_demo.txt'

In [ ]:
% time redistribute_gene_score(coef, mat, signal, out)